<a href="https://colab.research.google.com/github/linyuehzzz/hedetniemi_distance/blob/master/hedetniemi_distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Hedetniemi Matrix Sum**
This code is used to implement the [Hedetniemi Matrix Sum](https://deepblue.lib.umich.edu/handle/2027.42/59763).  
Yue Lin (lin.3326 at osu.edu)  
Created: 5/6/2020

#### **Generate graph data** 

In [0]:
## [node i, node j, distance between node i and j]
## using data from example 1: San Francisco Bay Area Graph of Time-Distances (in minutes)
data = [[1, 2, 30], [1, 4, 30], [1, 9, 40],
        [2, 3, 25], [2, 4, 40], [3, 4, 50],
        [4, 5, 30], [4, 6, 20], [5, 7, 25],
        [6, 7, 20], [6, 9, 20], [7, 8, 25],
        [8, 9, 20]]

#### **Implementation 1: list** 

Construct distance matrix

In [114]:
from timeit import default_timer

def distance_matrix(graph):
  ## calculate number of nodes
  n = max([g[1] for g in graph])

  ## calculate distance matrix
  INF = float('inf')
  dist_mtx = [[INF] * n for i in range(n)]
  for g in graph:
    i = g[0] - 1
    j = g[1] - 1
    d = g[2]
    dist_mtx[i][j] = d
    dist_mtx[j][i] = d

  ## set diagonal to 0
  for i in range(n):
    dist_mtx[i][i] = 0
 
  return dist_mtx, n


## print time costs
start = default_timer()
dist_mtx, n = distance_matrix(data)
stop = default_timer()
print('Time: ', stop - start)

## print distance matrix
print("Distance matrix: ")
for line in dist_mtx:
  print(line)

Time:  8.185199840227142e-05
Distance matrix: 
[0, 30, inf, 30, inf, inf, inf, inf, 40]
[30, 0, 25, 40, inf, inf, inf, inf, inf]
[inf, 25, 0, 50, inf, inf, inf, inf, inf]
[30, 40, 50, 0, 30, 20, inf, inf, inf]
[inf, inf, inf, 30, 0, inf, 25, inf, inf]
[inf, inf, inf, 20, inf, 0, 20, inf, 20]
[inf, inf, inf, inf, 25, 20, 0, 25, inf]
[inf, inf, inf, inf, inf, inf, 25, 0, 20]
[40, inf, inf, inf, inf, 20, inf, 20, 0]


Calculate Hedetniemi Matrix Sum

In [115]:
from timeit import default_timer

def hede_distance(matrix, n):
  INF = float('inf')
  mtx_a_t = [[INF] * n for i in range(n)]
  mtx_a_t_1 = matrix

  p = True
  while p:
    for i in range(n):
      a = mtx_a_t_1[i]
      for j in range(n):
        b = [row[j] for row in matrix]
        mtx_a_t[i][j] = min([a[k] + b[k] for k in range(n)])
    
    if mtx_a_t == mtx_a_t_1:
      p =  False
    else:
      mtx_a_t_1 = mtx_a_t   
  
  return mtx_a_t


## print time costs
start = default_timer()
mtx_a_t = hede_distance(dist_mtx, n)
stop = default_timer()
print('Time: ', stop - start)

## print shortest path matrix
print("Shortest path matrix: ")
for line in mtx_a_t:
  print(line)

Time:  0.0004270609933882952
Shortest path matrix: 
[0, 30, 55, 30, 60, 50, 70, 60, 40]
[30, 0, 25, 40, 70, 60, 80, 90, 70]
[55, 25, 0, 50, 80, 70, 90, 115, 90]
[30, 40, 50, 0, 30, 20, 40, 60, 40]
[60, 70, 80, 30, 0, 45, 25, 50, 65]
[50, 60, 70, 20, 45, 0, 20, 40, 20]
[70, 80, 90, 40, 25, 20, 0, 25, 40]
[60, 90, 115, 60, 50, 40, 25, 0, 20]
[40, 70, 90, 40, 65, 20, 40, 20, 0]


#### **Implementation 2: numpy** 

Construct distance matrix

In [116]:
from timeit import default_timer
import numpy as np

def distance_matrix(graph):
  ## calculate number of nodes
  n = np.amax(graph[:,1])

  ## calculate distance matrix
  dist_mtx = np.full((n,n), np.inf)
  for g in graph:
    i = g[0] - 1
    j = g[1] - 1
    d = g[2]
    dist_mtx[i,j] = d
    dist_mtx[j,i] = d

  ## set diagonal to 0
  np.fill_diagonal(dist_mtx, 0)
 
  return dist_mtx, n


## print time costs
start = default_timer()
dist_mtx, n = distance_matrix(np.array(data))
stop = default_timer()
print('Time: ', stop - start)

## print distance matrix
print("Distance matrix: ")
for line in dist_mtx:
  print(line)

Time:  0.0006318369996733963
Distance matrix: 
[ 0. 30. inf 30. inf inf inf inf 40.]
[30.  0. 25. 40. inf inf inf inf inf]
[inf 25.  0. 50. inf inf inf inf inf]
[30. 40. 50.  0. 30. 20. inf inf inf]
[inf inf inf 30.  0. inf 25. inf inf]
[inf inf inf 20. inf  0. 20. inf 20.]
[inf inf inf inf 25. 20.  0. 25. inf]
[inf inf inf inf inf inf 25.  0. 20.]
[40. inf inf inf inf 20. inf 20.  0.]


Calculate Hedetniemi Matrix Sum

In [117]:
from timeit import default_timer
import numpy as np

def hede_distance(matrix, n):
  mtx_a_t = np.full((n,n), np.inf)
  mtx_a_t_1 = matrix

  p = True
  while p:
    for i in range(n):
      a = mtx_a_t_1[i]
      for j in range(n):
        b = matrix[:,j]
        mtx_a_t[i,j] = np.amin([a[k] + b[k] for k in range(n)])
    
    if np.array_equal(mtx_a_t, mtx_a_t_1):
      p =  False
    else:
      mtx_a_t_1 = mtx_a_t   
  
  return mtx_a_t


## print time costs
start = default_timer()
mtx_a_t = hede_distance(dist_mtx, n)
stop = default_timer()
print('Time: ', stop - start)

## print shortest path matrix
print("Shortest path matrix: ")
for line in mtx_a_t:
  print(line)

Time:  0.002320665997103788
Shortest path matrix: 
[ 0. 30. 55. 30. 60. 50. 70. 60. 40.]
[30.  0. 25. 40. 70. 60. 80. 90. 70.]
[ 55.  25.   0.  50.  80.  70.  90. 115.  90.]
[30. 40. 50.  0. 30. 20. 40. 60. 40.]
[60. 70. 80. 30.  0. 45. 25. 50. 65.]
[50. 60. 70. 20. 45.  0. 20. 40. 20.]
[70. 80. 90. 40. 25. 20.  0. 25. 40.]
[ 60.  90. 115.  60.  50.  40.  25.   0.  20.]
[40. 70. 90. 40. 65. 20. 40. 20.  0.]


#### **Implementation 3: tensorflow** 

In [0]:
!pip install -q tf-nightly

Construct distance matrix

In [119]:
from timeit import default_timer
import tensorflow as tf
import numpy as np

start = default_timer()

## convert graph
graph = tf.constant(data, dtype=tf.float32)
length = tf.dtypes.cast(tf.size(graph)/3, tf.int32)
graph = tf.gather(graph, tf.nn.top_k(-graph[:,0], k=length).indices)

## calculate number of nodes
n = tf.math.reduce_max(graph[:,1])
n = tf.dtypes.cast(n, tf.int32)

########### calculate distance matrix ###########
m = tf.shape(graph)[0]                    ## number of rows in graph
k = tf.constant(0)                        ## index for row in graph
i = tf.constant(0, dtype=tf.float32)      ## index for row in distance matrix
row = tf.fill([1, n], np.inf)

while tf.math.less(k, m):
  if tf.math.equal(tf.math.subtract(graph[k][0], 1), i):    
    p = tf.one_hot([tf.math.subtract(graph[k][1], 1)], depth=n, 
                   on_value=graph[k][2], off_value=np.inf, axis=-1)    
    row = tf.math.minimum(row, p, name=None)
    k = tf.math.add(k, 1)
  else:
    if tf.math.equal(i, 0):
      dist_mtx = row
    else:
      dist_mtx = tf.concat([dist_mtx, row], 0) 
    i = tf.math.add(i, 1)
    row = tf.fill([1, n], np.inf)

dist_mtx = tf.concat([dist_mtx, row], 0)
row = tf.constant(tf.fill([1, n], np.inf))
while tf.math.less(tf.dtypes.cast(i, tf.int32), tf.math.subtract(n, 1)):  
  dist_mtx = tf.concat([dist_mtx, row], 0)
  i = tf.math.add(i, 1)

## add transposed matrix
dist_mtx_tr = tf.transpose(dist_mtx)
dist_mtx = tf.math.minimum(dist_mtx_tr, dist_mtx, name=None)

## set diagonal to 0
dist_mtx = tf.linalg.set_diag(dist_mtx, tf.zeros([n], tf.float32), name=None)

## print time costs
stop = default_timer()
print('Time: ', stop - start)

## print distance matrix
print("Distance matrix: ")
print(dist_mtx)

Time:  0.025642478998634033
Distance matrix: 
tf.Tensor(
[[ 0. 30. inf 30. inf inf inf inf 40.]
 [30.  0. 25. 40. inf inf inf inf inf]
 [inf 25.  0. 50. inf inf inf inf inf]
 [30. 40. 50.  0. 30. 20. inf inf inf]
 [inf inf inf 30.  0. inf 25. inf inf]
 [inf inf inf 20. inf  0. 20. inf 20.]
 [inf inf inf inf 25. 20.  0. 25. inf]
 [inf inf inf inf inf inf 25.  0. 20.]
 [40. inf inf inf inf 20. inf 20.  0.]], shape=(9, 9), dtype=float32)
